In [3]:
# Import required libraries
from bs4 import BeautifulSoup
import requests
import string
import csv

## Get Static Fight Stats

In [33]:
from bs4 import BeautifulSoup
import requests

def get_hometown_and_trainsoutof(name):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Cache-Control': 'max-age=0'
    }

    name_parts = name.split(' ')
    first_name, middle_name, last_name = None, None, None

    if len(name_parts) == 2:
        first_name, last_name = name_parts
    elif len(name_parts) == 3:
        first_name, middle_name, last_name = name_parts

    name_variations = set()
    name_variations.add(name.lower())
    if middle_name:
        name_variations.add(f"{first_name} {last_name}".lower())
        name_variations.add(f"{middle_name} {last_name}".lower())
        name_variations.add(f"{first_name} {middle_name} {last_name}".lower())
        name_variations.add(f"{last_name} {first_name} {middle_name}".lower())
    else:
        name_variations.add(f"{last_name} {first_name}".lower())

    url = 'https://www.sherdog.com/stats/fightfinder?SearchTxt=' + name.replace(' ', '+')
    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.text, 'lxml')

    try:
        fighters = soup.find_all('table', class_='new_table fightfinder_result')[0].find_all('a')
    except IndexError:
        return "Fighter not found", "Fighter not found"

    fighter_link = None
    nickname = soup.find_all('table', class_='new_table fightfinder_result')[0].find_all('td')[7].text.strip().strip('"').lower()

    last_name_nickname = f"{nickname} {last_name}".lower()
    first_name_nickname = f"{first_name} {nickname}".lower()
    found_name = None
    for fighter in fighters:
        if fighter.text.lower() in name_variations:
            fighter_link = fighter.get('href')
            
            break
        elif last_name_nickname in name_variations:
            fighter_link = fighter.get('href')
            found_name = fighter.text  # Store the found name
            break
        elif first_name_nickname in name_variations:
            fighter_link = fighter.get('href')
            found_name = fighter.text  # Store the found name
            break

    if fighter_link is None:
        return "Fighter not found", "Fighter not found"

    fighter_url = 'https://www.sherdog.com' + fighter_link
    source2 = requests.get(fighter_url, headers=headers)
    soup2 = BeautifulSoup(source2.text, 'lxml')

    try:
        hometown = soup2.find_all('span', class_='locality')[0].text.strip()
    except IndexError:
        hometown = "Hometown Not found"

    try:
        trains_out_of = soup2.find_all('a', class_='association')[0].text.strip()
    except IndexError:
        trains_out_of = "Trains out of Not found"

    return found_name, hometown, trains_out_of

# Test the function
print(get_hometown_and_trainsoutof('Cyborg Abreu'))


('Roberto de Abreu Filho', 'Campo Grande, Mato Grosso do Sul', 'Fight Sports')


In [32]:
# Initialize the CSV file for writing
with open('ufc_fighters_static.csv', 'w', newline='') as csvfile:
    # Define the column headers for the CSV
    fieldnames = ['Name', 'Height', 'Reach', 'Stance', 'DOB', 'ID', 'Hometown', 'Trains_Out_Of']
    
    # Initialize the CSV writer
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Write the header row to the CSV
    writer.writeheader()

    # Generate a list of lowercase alphabets to iterate through
    alphabets = list(string.ascii_lowercase)

    # Loop through each alphabet letter
    for letter in alphabets:
        # Fetch the fighter list page for the current alphabet letter
        source = requests.get(f'http://www.ufcstats.com/statistics/fighters?char={letter}&page=all').text
        soup = BeautifulSoup(source, "lxml")

        # Loop through each fighter entry on the page
        for fighter in soup.find_all('tr', attrs={'class': 'b-statistics__table-row'}):
            a_href = fighter.find('a')
            
            # Check if the fighter entry has a URL
            if a_href is not None:
                # Fetch the fighter's individual stats page
                source2 = requests.get(a_href.get("href")).text
                
                # Extract the fighter ID from the URL
                ID = a_href.get("href").split('/')[-1].strip()
                
                soup2 = BeautifulSoup(source2, "lxml")

                # Extract and store the fighter's name
                name = soup2.find('span', {'class': 'b-content__title-highlight'}).text.strip()

                # Initialize a dictionary to hold the fighter's stats
                fighter_stats = {'Name': name, 'ID': ID}

                # Extract and store other stats like Height, Reach, Stance, and DOB
                stats = soup2.find_all('li', attrs={'class': 'b-list__box-list-item'})
                for stat in stats:
                    cleaned_stat = [string.strip() for string in stat.text.split(":")]
                    if len(cleaned_stat) > 1:
                        key = cleaned_stat[0]
                        value = cleaned_stat[1]
                        fighter_stats[key] = value

                # Extract and store the fighter's hometown and trains out of
                try:
                    found_name, hometown, trains_out_of = get_hometown_and_trainsoutof(name)
                    if found_name:
                        fighter_stats['Name'] = found_name  # Update the name with the one found on Sherdog
                    fighter_stats['Hometown'] = hometown
                    fighter_stats['Trains_Out_Of'] = trains_out_of
                except:
                    fighter_stats['Hometown'] = "Fighter not found"
                    fighter_stats['Trains_Out_Of'] = "Fighter not found"



                # Filter out keys not in fieldnames
                filtered_fighter_stats = {k: v for k, v in fighter_stats.items() if k in fieldnames}
                writer.writerow(filtered_fighter_stats)
               
                
                



### Main Scraping Functions

In [108]:
"""
RUN THIS BOX TO SEE THE CURRENT RESULTS AND WHAT THE DATA SHOULD LOOK LIKE.

Eventually, this collection of functions should be under a FightData object 
-> Then we can just call FightData.scrape_ufc_stats() as well as update the columns 
after every new fight -> make this a cron job that runs every sat night after the fights.

Feel free to optimize this if it looks a little chunky.

Data scraped should be in the following format:
[col1_title, col2_title, col3_title, ...]
[r1c1_data,  r1c2_data,  r1c3_data,  ...]
[r2c1_data,  r2c2_data,  r2c3_data,  ...]
[r3c1_data,  r3c2_data,  r3c3_data,  ...]
[r4c1_data,  r4c2_data,  r4c3_data,  ...]
...

The data will then be put into a CSV file.
"""

# Add to these columns with new data scraped
columns = ['Fight Night Title', 'date', 'location', 'Fighter A', 'Fighter A ID', 'Fighter B', 'Fighter B ID', 'Division', 
           'Outcome Method', 'Outcome Round', 'Outcome Time', 'Outcome Format', 
           'Referee', 'Outcome Detail', 
           'Fighter A Round 1 KD', 'Fighter A Round 1 SIG STR Landed', 'Fighter A Round 1 SIG STR Attempted', 
           'Fighter A Round 1 SIG STR %', 'Fighter A Round 1 TOTAL STR Landed', 'Fighter A Round 1 TOTAL STR Attempted', 
           'Fighter A Round 1 TD Landed', 'Fighter A Round 1 Attempted', 'Fighter A Round 1 TD %', 
           'Fighter A Round 1 SUB ATT', 'Fighter A Round 1 REV', 'Fighter A Round 1 CTRL',
           'Fighter A Round 2 KD', 'Fighter A Round 2 SIG STR Landed', 'Fighter A Round 2 SIG STR Attempted', 
           'Fighter A Round 2 SIG STR %', 'Fighter A Round 2 TOTAL STR Landed', 'Fighter A Round 2 TOTAL STR Attempted', 
           'Fighter A Round 2 TD Landed', 'Fighter A Round 2 Attempted', 'Fighter A Round 2 TD %', 
           'Fighter A Round 2 SUB ATT', 'Fighter A Round 2 REV', 'Fighter A Round 2 CTRL',
           'Fighter A Round 3 KD', 'Fighter A Round 3 SIG STR Landed', 'Fighter A Round 3 SIG STR Attempted', 
           'Fighter A Round 3 SIG STR %', 'Fighter A Round 3 TOTAL STR Landed', 'Fighter A Round 3 TOTAL STR Attempted', 
           'Fighter A Round 3 TD Landed', 'Fighter A Round 3 Attempted', 'Fighter A Round 3 TD %', 
           'Fighter A Round 3 SUB ATT', 'Fighter A Round 3 REV', 'Fighter A Round 3 CTRL',
           'Fighter A Round 4 KD', 'Fighter A Round 4 SIG STR Landed', 'Fighter A Round 4 SIG STR Attempted', 
           'Fighter A Round 4 SIG STR %', 'Fighter A Round 4 TOTAL STR Landed', 'Fighter A Round 4 TOTAL STR Attempted', 
           'Fighter A Round 4 TD Landed', 'Fighter A Round 4 Attempted', 'Fighter A Round 4 TD %', 
           'Fighter A Round 4 SUB ATT', 'Fighter A Round 4 REV', 'Fighter A Round 4 CTRL',
           'Fighter A Round 5 KD', 'Fighter A Round 5 SIG STR Landed', 'Fighter A Round 5 SIG STR Attempted', 
           'Fighter A Round 5 SIG STR %', 'Fighter A Round 5 TOTAL STR Landed', 'Fighter A Round 5 TOTAL STR Attempted', 
           'Fighter A Round 5 TD Landed', 'Fighter A Round 5 Attempted', 'Fighter A Round 5 TD %', 
           'Fighter A Round 5 SUB ATT', 'Fighter A Round 5 REV', 'Fighter A Round 5 CTRL',
           'Fighter A Total KD', 'Fighter A Total SIG STR Landed', 'Fighter A Total SIG STR Attempted', 
           'Fighter A Total SIG STR %', 'Fighter A Total TOTAL STR Landed', 'Fighter A Total TOTAL STR Attempted', 
           'Fighter A Total TD Landed', 'Fighter A Total Attempted', 'Fighter A Total TD %', 
           'Fighter A Total SUB ATT', 'Fighter A Total REV', 'Fighter A Total CTRL',
           'Fighter B Round 1 KD', 'Fighter B Round 1 SIG STR Landed', 'Fighter B Round 1 SIG STR Attempted', 
           'Fighter B Round 1 SIG STR %', 'Fighter B Round 1 TOTAL STR Landed', 'Fighter B Round 1 TOTAL STR Attempted', 
           'Fighter B Round 1 TD Landed', 'Fighter B Round 1 Attempted', 'Fighter B Round 1 TD %', 
           'Fighter B Round 1 SUB ATT', 'Fighter B Round 1 REV', 'Fighter B Round 1 CTRL',
           'Fighter B Round 2 KD', 'Fighter B Round 2 SIG STR Landed', 'Fighter B Round 2 SIG STR Attempted', 
           'Fighter B Round 2 SIG STR %', 'Fighter B Round 2 TOTAL STR Landed', 'Fighter B Round 2 TOTAL STR Attempted', 
           'Fighter B Round 2 TD Landed', 'Fighter B Round 2 Attempted', 'Fighter B Round 2 TD %', 
           'Fighter B Round 2 SUB ATT', 'Fighter B Round 2 REV', 'Fighter B Round 2 CTRL',
           'Fighter B Round 3 KD', 'Fighter B Round 3 SIG STR Landed', 'Fighter B Round 3 SIG STR Attempted', 
           'Fighter B Round 3 SIG STR %', 'Fighter B Round 3 TOTAL STR Landed', 'Fighter B Round 3 TOTAL STR Attempted', 
           'Fighter B Round 3 TD Landed', 'Fighter B Round 3 Attempted', 'Fighter B Round 3 TD %', 
           'Fighter B Round 3 SUB ATT', 'Fighter B Round 3 REV', 'Fighter B Round 3 CTRL',
           'Fighter B Round 4 KD', 'Fighter B Round 4 SIG STR Landed', 'Fighter B Round 4 SIG STR Attempted', 
           'Fighter B Round 4 SIG STR %', 'Fighter B Round 4 TOTAL STR Landed', 'Fighter B Round 4 TOTAL STR Attempted', 
           'Fighter B Round 4 TD Landed', 'Fighter B Round 4 Attempted', 'Fighter B Round 4 TD %', 
           'Fighter B Round 4 SUB ATT', 'Fighter B Round 4 REV', 'Fighter B Round 4 CTRL',
           'Fighter B Round 5 KD', 'Fighter B Round 5 SIG STR Landed', 'Fighter B Round 5 SIG STR Attempted', 
           'Fighter B Round 5 SIG STR %', 'Fighter B Round 5 TOTAL STR Landed', 'Fighter B Round 5 TOTAL STR Attempted', 
           'Fighter B Round 5 TD Landed', 'Fighter B Round 5 Attempted', 'Fighter B Round 5 TD %', 
           'Fighter B Round 5 SUB ATT', 'Fighter B Round 5 REV', 'Fighter B Round 5 CTRL',
           'Fighter B Total KD', 'Fighter B Total SIG STR Landed', 'Fighter B Total SIG STR Attempted', 
           'Fighter B Total SIG STR %', 'Fighter B Total TOTAL STR Landed', 'Fighter B Total TOTAL STR Attempted', 
           'Fighter B Total TD Landed', 'Fighter B Total Attempted', 'Fighter B Total TD %', 
           'Fighter B Total SUB ATT', 'Fighter B Total REV', 'Fighter B Total CTRL',
           'Fighter A Round 1 Head Shots Landed', 'Fighter A Round 1 Head Shots Attempted', 
           'Fighter A Round 1 Body Shots Landed', 'Fighter A Round 1 Body Shots Attempted', 
           'Fighter A Round 1 Leg Shots Landed', 'Fighter A Round 1 Leg Shots Attempted', 
           'Fighter A Round 1 Distance Shots Landed', 'Fighter A Round 1 Distance Shots Attempted', 
           'Fighter A Round 1 Clinch Landed', 'Fighter A Round 1 Clinch Attempted', 
           'Fighter A Round 1 Ground Landed', 'Fighter A Round 1 Ground Attempted',
           'Fighter A Round 2 Head Shots Landed', 'Fighter A Round 2 Head Shots Attempted', 
           'Fighter A Round 2 Body Shots Landed', 'Fighter A Round 2 Body Shots Attempted', 
           'Fighter A Round 2 Leg Shots Landed', 'Fighter A Round 2 Leg Shots Attempted', 
           'Fighter A Round 2 Distance Shots Landed', 'Fighter A Round 2 Distance Shots Attempted', 
           'Fighter A Round 2 Clinch Landed', 'Fighter A Round 2 Clinch Attempted', 
           'Fighter A Round 2 Ground Landed', 'Fighter A Round 2 Ground Attempted',
           'Fighter A Round 3 Head Shots Landed', 'Fighter A Round 3 Head Shots Attempted', 
           'Fighter A Round 3 Body Shots Landed', 'Fighter A Round 3 Body Shots Attempted', 
           'Fighter A Round 3 Leg Shots Landed', 'Fighter A Round 3 Leg Shots Attempted', 
           'Fighter A Round 3 Distance Shots Landed', 'Fighter A Round 3 Distance Shots Attempted', 
           'Fighter A Round 3 Clinch Landed', 'Fighter A Round 3 Clinch Attempted', 
           'Fighter A Round 3 Ground Landed', 'Fighter A Round 3 Ground Attempted',
           'Fighter A Round 4 Head Shots Landed', 'Fighter A Round 4 Head Shots Attempted', 
           'Fighter A Round 4 Body Shots Landed', 'Fighter A Round 4 Body Shots Attempted', 
           'Fighter A Round 4 Leg Shots Landed', 'Fighter A Round 4 Leg Shots Attempted', 
           'Fighter A Round 4 Distance Shots Landed', 'Fighter A Round 4 Distance Shots Attempted', 
           'Fighter A Round 4 Clinch Landed', 'Fighter A Round 4 Clinch Attempted', 
           'Fighter A Round 4 Ground Landed', 'Fighter A Round 4 Ground Attempted',
           'Fighter A Round 5 Head Shots Landed', 'Fighter A Round 5 Head Shots Attempted', 
           'Fighter A Round 5 Body Shots Landed', 'Fighter A Round 5 Body Shots Attempted', 
           'Fighter A Round 5 Leg Shots Landed', 'Fighter A Round 5 Leg Shots Attempted', 
           'Fighter A Round 5 Distance Shots Landed', 'Fighter A Round 5 Distance Shots Attempted', 
           'Fighter A Round 5 Clinch Landed', 'Fighter A Round 5 Clinch Attempted', 
           'Fighter A Round 5 Ground Landed', 'Fighter A Round 5 Ground Attempted',
           'Fighter A Total Head Shots Landed', 'Fighter A Total Head Shots Attempted', 
           'Fighter A Total Body Shots Landed', 'Fighter A Total Body Shots Attempted', 
           'Fighter A Total Leg Shots Landed', 'Fighter A Total Leg Shots Attempted', 
           'Fighter A Total Distance Shots Landed', 'Fighter A Total Distance Shots Attempted', 
           'Fighter A Total Clinch Landed', 'Fighter A Total Clinch Attempted', 
           'Fighter A Total Ground Landed', 'Fighter A Total Ground Attempted',
           'Fighter B Round 1 Head Shots Landed', 'Fighter B Round 1 Head Shots Attempted', 
           'Fighter B Round 1 Body Shots Landed', 'Fighter B Round 1 Body Shots Attempted', 
           'Fighter B Round 1 Leg Shots Landed', 'Fighter B Round 1 Leg Shots Attempted', 
           'Fighter B Round 1 Distance Shots Landed', 'Fighter B Round 1 Distance Shots Attempted', 
           'Fighter B Round 1 Clinch Landed', 'Fighter B Round 1 Clinch Attempted', 
           'Fighter B Round 1 Ground Landed', 'Fighter B Round 1 Ground Attempted',
           'Fighter B Round 2 Head Shots Landed', 'Fighter B Round 2 Head Shots Attempted', 
           'Fighter B Round 2 Body Shots Landed', 'Fighter B Round 2 Body Shots Attempted', 
           'Fighter B Round 2 Leg Shots Landed', 'Fighter B Round 2 Leg Shots Attempted', 
           'Fighter B Round 2 Distance Shots Landed', 'Fighter B Round 2 Distance Shots Attempted', 
           'Fighter B Round 2 Clinch Landed', 'Fighter B Round 2 Clinch Attempted', 
           'Fighter B Round 2 Ground Landed', 'Fighter B Round 2 Ground Attempted',
           'Fighter B Round 3 Head Shots Landed', 'Fighter B Round 3 Head Shots Attempted', 
           'Fighter B Round 3 Body Shots Landed', 'Fighter B Round 3 Body Shots Attempted', 
           'Fighter B Round 3 Leg Shots Landed', 'Fighter B Round 3 Leg Shots Attempted', 
           'Fighter B Round 3 Distance Shots Landed', 'Fighter B Round 3 Distance Shots Attempted', 
           'Fighter B Round 3 Clinch Landed', 'Fighter B Round 3 Clinch Attempted', 
           'Fighter B Round 3 Ground Landed', 'Fighter B Round 3 Ground Attempted',
           'Fighter B Round 4 Head Shots Landed', 'Fighter B Round 4 Head Shots Attempted', 
           'Fighter B Round 4 Body Shots Landed', 'Fighter B Round 4 Body Shots Attempted', 
           'Fighter B Round 4 Leg Shots Landed', 'Fighter B Round 4 Leg Shots Attempted', 
           'Fighter B Round 4 Distance Shots Landed', 'Fighter B Round 4 Distance Shots Attempted', 
           'Fighter B Round 4 Clinch Landed', 'Fighter B Round 4 Clinch Attempted', 
           'Fighter B Round 4 Ground Landed', 'Fighter B Round 4 Ground Attempted',
           'Fighter B Round 5 Head Shots Landed', 'Fighter B Round 5 Head Shots Attempted', 
           'Fighter B Round 5 Body Shots Landed', 'Fighter B Round 5 Body Shots Attempted', 
           'Fighter B Round 5 Leg Shots Landed', 'Fighter B Round 5 Leg Shots Attempted', 
           'Fighter B Round 5 Distance Shots Landed', 'Fighter B Round 5 Distance Shots Attempted', 
           'Fighter B Round 5 Clinch Landed', 'Fighter B Round 5 Clinch Attempted', 
           'Fighter B Round 5 Ground Landed', 'Fighter B Round 5 Ground Attempted',
           'Fighter B Total Head Shots Landed', 'Fighter B Total Head Shots Attempted', 
           'Fighter B Total Body Shots Landed', 'Fighter B Total Body Shots Attempted', 
           'Fighter B Total Leg Shots Landed', 'Fighter B Total Leg Shots Attempted', 
           'Fighter B Total Distance Shots Landed', 'Fighter B Total Distance Shots Attempted', 
           'Fighter B Total Clinch Landed', 'Fighter B Total Clinch Attempted', 
           'Fighter B Total Ground Landed', 'Fighter B Total Ground Attempted',
        ]

UFC_STATS_URL = 'http://www.ufcstats.com/statistics/events/completed?page=all'

def scrape_ufc_stats():
    # Open the CSV file in append mode
    with open('ufc_men_stats_by_fight.csv', 'a', newline='') as men_csv_file, open('ufc_women_stats_by_fight.csv', 'a', newline='') as women_csv_file:
        writer1 = csv.writer(men_csv_file)
        writer2 = csv.writer(women_csv_file)

        # Write the header only if the file is empty
        if men_csv_file.tell() == 0:
            writer1.writerow(columns)  

        if women_csv_file.tell() == 0:
            writer2.writerow(columns)

        #scrape the data
        source_main = requests.get(UFC_STATS_URL).text
        soup_main = BeautifulSoup(source_main, "lxml")
        fights_main = soup_main.find_all('tr', attrs={'class': 'b-statistics__table-row'})
        # Loop through each fight card     
        for i in range(len(fights_main) - 1, -1, -1):
            fight_night = fights_main[3]
            fight_night_ahref = fight_night.find('a')
            
            if fight_night_ahref is not None:
                fight_night_title = fight_night_ahref.text.strip()
                fight_night_link = fight_night_ahref.get("href")

                # Create an empty list to hold the data for each fight
                men_data = []
                women_data = []

                get_fights(fight_night_link, men_data, women_data, fight_night_title)

                # Write the newly scraped data to the CSV
                for row in men_data:
                    writer1.writerow(row)

                for row in women_data:
                    writer2.writerow(row)
            
            break

# Gets the individual fights from the fight night
def get_fights(link, men_data, women_data, fight_night_title):
    source_fight_night = requests.get(link).text
    soup_fight_night = BeautifulSoup(source_fight_night, "lxml")
    fight_night_fights = soup_fight_night.find_all('tr', attrs={'class': 'b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click'})
    date, location = get_fight_date_and_location(soup_fight_night)
    for j in range(len(fight_night_fights)-1, -1, -1):
        fight = fight_night_fights[j]
        fight_ahref = fight.find('a')
        if fight_ahref is not None:
            fight_link = fight_ahref.get("href")
            fighters_data = get_fighters(fight_link)
            fight_overview_data = get_fight_overview(fight_link)
            # Add more functions below that scrape certain sections of the ufcstats page
            # Totals (KD, Sig. Str, Sig. Str. %, Total Strikes, Takedowns, Takedown %, Sub. Att, Rev., Crtl.)
            # ^ Per round as well as we want the most possible data
            # Add Significant Strikes by Target (Head, Body, Leg) as well

            fight_totals = get_fight_totals(fight_link, fight_overview_data[2])
            fight_totals_tuple = tuple(element for tupl in fight_totals for element in tupl)
            fight_sig_strikes = get_fight_sig_strikes(fight_link, fight_overview_data[2])
            fight_sig_strikes_tuple = tuple(element for tupl in fight_sig_strikes for element in tupl)
            
            # check if the fight was a women fight and write data to correct location
            women_fight = any("women" in p.text.strip().lower() for p in fight.findChildren('p'))
            if women_fight:
                women_data.append([fight_night_title] + [date] + [location] + list(fighters_data) + list(fight_overview_data) + list(fight_totals_tuple) + list(fight_sig_strikes_tuple))
            else:
                men_data.append([fight_night_title] + [date] + [location] + list(fighters_data) + list(fight_overview_data) + list(fight_totals_tuple) + list(fight_sig_strikes_tuple))
            #display(data)

        # Remove after development
        # break

# Gets the fighters' names and links
def get_fighters(link):
    source_fight = requests.get(link).text
    soup_fight = BeautifulSoup(source_fight, "lxml")
    fighters = soup_fight.find_all('a', class_='b-link b-fight-details__person-link')

    fighter_a = fighters[0]
    fighter_a_name = fighter_a.text.strip()
    fighter_a_id = fighter_a.get("href").split('/')[-1].strip()

    fighter_b = fighters[1]
    fighter_b_name = fighter_b.text.strip()
    fighter_b_id = fighter_b.get("href").split('/')[-1].strip()

    return fighter_a_name, fighter_a_id, fighter_b_name, fighter_b_id

# Gets the overview box information
def get_fight_overview(link):
    source = requests.get(link).text
    soup = BeautifulSoup(source, "lxml")

    details = soup.find_all('i', attrs={'class' : 'b-fight-details__text-item'})

    division = ' '.join(soup.find('i', attrs={'class' : 'b-fight-details__fight-title'}).text.strip().rsplit((' ', 1)[0])[:-1])
    outcome_method = soup.find('i', attrs={'style' : 'font-style: normal'}).text.strip()
    outcome_round = details[0].text.strip().split(' ')[-1]
    outcome_time = details[1].text.strip().split(' ')[-1]
    outcome_format = details[2].text.strip().split(' ')[-1]
    referee = details[3].text.strip().split(' ')[-1]
    outcome_detail = ' '.join([word for word in soup.find_all(True, attrs={'class' : 'b-fight-details__text'})[-1].text.strip().split(' ') if word.strip()][1:])

    return division, outcome_method, outcome_round, outcome_time, outcome_format, referee, outcome_detail

def get_fight_totals(link, round):
    source = requests.get(link).text
    soup = BeautifulSoup(source, "lxml")
    stats = soup.find_all('section', attrs={'class' : 'b-fight-details__section js-fight-section'})
    player_a_total_stats = player_a_round_1_stats = player_a_round_2_stats = player_a_round_3_stats = player_a_round_4_stats = player_a_round_5_stats = (None, None, None, None, None, None, None, None, None, None, None, None)
    player_b_total_stats = player_b_round_1_stats = player_b_round_2_stats = player_b_round_3_stats = player_b_round_4_stats = player_b_round_5_stats = (None, None, None, None, None, None, None, None, None, None, None, None)
    if(len(stats) > 1):
        total_stats = stats[1].find('table').find_all('tr', attrs={'class' : 'b-fight-details__table-row'})[1].find_all('p', attrs={'class' : 'b-fight-details__table-text'})
       
        for i in range(0, int(round)):
            if i == 0: 
                totals_per_round_1 = stats[2].find('table').find_all('tr', attrs={'class' : 'b-fight-details__table-row'})[1].find_all('p', attrs={'class' : 'b-fight-details__table-text'})
                player_a_round_1_stats = get_total_fight_stats(totals_per_round_1)[0]
                player_b_round_1_stats = get_total_fight_stats(totals_per_round_1)[1]
            elif i == 1:
                totals_per_round_2 = stats[2].find('table').find_all('tr', attrs={'class' : 'b-fight-details__table-row'})[2].find_all('p', attrs={'class' : 'b-fight-details__table-text'})
                player_a_round_2_stats = get_total_fight_stats(totals_per_round_2)[0]
                player_b_round_2_stats = get_total_fight_stats(totals_per_round_2)[1]
            elif i == 2:
                totals_per_round_3 = stats[2].find('table').find_all('tr', attrs={'class' : 'b-fight-details__table-row'})[3].find_all('p', attrs={'class' : 'b-fight-details__table-text'})
                player_a_round_3_stats = get_total_fight_stats(totals_per_round_3)[0]
                player_b_round_3_stats = get_total_fight_stats(totals_per_round_3)[1]
            elif i == 3:
                totals_per_round_4 = stats[2].find('table').find_all('tr', attrs={'class' : 'b-fight-details__table-row'})[4].find_all('p', attrs={'class' : 'b-fight-details__table-text'})
                player_a_round_4_stats = get_total_fight_stats(totals_per_round_4)[0]
                player_b_round_4_stats = get_total_fight_stats(totals_per_round_4)[1]
            elif i == 4:
                totals_per_round_5 = stats[2].find('table').find_all('tr', attrs={'class' : 'b-fight-details__table-row'})[5].find_all('p', attrs={'class' : 'b-fight-details__table-text'})
                player_a_round_5_stats = get_total_fight_stats(totals_per_round_5)[0]
                player_b_round_5_stats = get_total_fight_stats(totals_per_round_5)[1]

        player_a_total_stats = get_total_fight_stats(total_stats)[0]
        player_b_total_stats = get_total_fight_stats(total_stats)[1]

    return player_a_round_1_stats, player_a_round_2_stats, player_a_round_3_stats, player_a_round_4_stats, player_a_round_5_stats, player_a_total_stats, player_b_round_1_stats, player_b_round_2_stats, player_b_round_3_stats, player_b_round_4_stats, player_b_round_5_stats, player_b_total_stats, 

def get_total_fight_stats(totals):
    fighter_a_kd = totals[2].text.strip()
    fighter_b_kd = totals[3].text.strip()
    fighter_a_sig_str_landed = totals[4].text.split("of")[0].strip()
    fighter_a_sig_str_attempted = totals[4].text.split("of")[1].strip()
    fighter_b_sig_str_landed = totals[5].text.split("of")[0].strip()
    fighter_b_sig_str_attempted = totals[5].text.split("of")[1].strip()
    fighter_a_sig_str_per = totals[6].text.strip()
    fighter_b_sig_str_per = totals[7].text.strip()
    fighter_a_total_str_landed = totals[8].text.split("of")[0].strip()
    fighter_a_total_str_attempted = totals[8].text.split("of")[1].strip()
    fighter_b_total_str_landed = totals[9].text.split("of")[0].strip()
    fighter_b_total_str_attempted = totals[9].text.split("of")[1].strip()
    fighter_a_total_td_landed = totals[10].text.split("of")[0].strip()
    fighter_a_total_td_attempted = totals[10].text.split("of")[1].strip()
    fighter_b_total_td_landed = totals[11].text.split("of")[0].strip()
    fighter_b_total_td_attempted = totals[11].text.split("of")[1].strip()
    fighter_a_total_td_per = totals[12].text.strip()
    fighter_b_total_td_per = totals[13].text.strip()
    fighter_a_total_sub_att = totals[14].text.strip()
    fighter_b_total_sub_att = totals[15].text.strip()
    fighter_a_total_rev = totals[16].text.strip()
    fighter_b_total_rev = totals[17].text.strip()
    fighter_a_total_ctrl = totals[18].text.strip()
    fighter_b_total_ctrl = totals[19].text.strip()

    fighter_a = fighter_a_kd, fighter_a_sig_str_landed, fighter_a_sig_str_attempted, fighter_a_sig_str_per, fighter_a_total_str_landed, fighter_a_total_str_attempted, fighter_a_total_td_landed, fighter_a_total_td_attempted, fighter_a_total_td_per, fighter_a_total_sub_att, fighter_a_total_rev, fighter_a_total_ctrl
    fighter_b = fighter_b_kd, fighter_b_sig_str_landed, fighter_b_sig_str_attempted, fighter_b_sig_str_per, fighter_b_total_str_landed, fighter_b_total_str_attempted, fighter_b_total_td_landed, fighter_b_total_td_attempted, fighter_b_total_td_per, fighter_b_total_sub_att, fighter_b_total_rev, fighter_b_total_ctrl

    return fighter_a, fighter_b

def get_fight_sig_strikes(link, round):
    source = requests.get(link).text
    soup = BeautifulSoup(source, "lxml")
    stats = soup.find_all('section', attrs={'class' : 'b-fight-details__section js-fight-section'})
    player_a_total_stats = player_a_round_1_stats = player_a_round_2_stats = player_a_round_3_stats = player_a_round_4_stats = player_a_round_5_stats = (None, None, None, None, None, None, None, None, None, None, None, None)
    player_b_total_stats = player_b_round_1_stats = player_b_round_2_stats = player_b_round_3_stats = player_b_round_4_stats = player_b_round_5_stats = (None, None, None, None, None, None, None, None, None, None, None, None)
    if len(soup.find_all('table', attrs={'style': 'width: 745px'})) > 1:
        total_stats = soup.find_all('table', attrs={'style': 'width: 745px'})[1].find_all('tr', attrs={'class' : 'b-fight-details__table-row'})[1].find_all('p', attrs={'class' : 'b-fight-details__table-text'})
        for i in range(0, int(round)):
            if i == 0:
                totals_per_round_1 = stats[4].find('table').find_all('tr', attrs={'class' : 'b-fight-details__table-row'})[1].find_all('p', attrs={'class' : 'b-fight-details__table-text'})
                player_a_round_1_stats = get_fight_sig_stats(totals_per_round_1)[0]
                player_b_round_1_stats = get_fight_sig_stats(totals_per_round_1)[1]
            elif i == 1:
                totals_per_round_2 = stats[4].find('table').find_all('tr', attrs={'class' : 'b-fight-details__table-row'})[2].find_all('p', attrs={'class' : 'b-fight-details__table-text'})
                player_a_round_2_stats = get_fight_sig_stats(totals_per_round_2)[0]
                player_b_round_2_stats = get_fight_sig_stats(totals_per_round_2)[1]
            elif i == 2:
                totals_per_round_3 = stats[4].find('table').find_all('tr', attrs={'class' : 'b-fight-details__table-row'})[3].find_all('p', attrs={'class' : 'b-fight-details__table-text'})
                player_a_round_3_stats = get_fight_sig_stats(totals_per_round_3)[0]
                player_b_round_3_stats = get_fight_sig_stats(totals_per_round_3)[1]
            elif i == 3:
                totals_per_round_4 = stats[4].find('table').find_all('tr', attrs={'class' : 'b-fight-details__table-row'})[4].find_all('p', attrs={'class' : 'b-fight-details__table-text'})
                player_a_round_4_stats = get_fight_sig_stats(totals_per_round_4)[0]
                player_b_round_4_stats = get_fight_sig_stats(totals_per_round_4)[1]
            elif i == 4:
                totals_per_round_5 = stats[4].find('table').find_all('tr', attrs={'class' : 'b-fight-details__table-row'})[5].find_all('p', attrs={'class' : 'b-fight-details__table-text'})
                player_a_round_5_stats = get_fight_sig_stats(totals_per_round_5)[0]
                player_b_round_5_stats = get_fight_sig_stats(totals_per_round_5)[1]
        
        player_a_total_stats = get_fight_sig_stats(total_stats)[0]
        player_b_total_stats = get_fight_sig_stats(total_stats)[1]

    return player_a_round_1_stats, player_a_round_2_stats, player_a_round_3_stats, player_a_round_4_stats, player_a_round_5_stats, player_a_total_stats, player_b_round_1_stats, player_b_round_2_stats, player_b_round_3_stats, player_b_round_4_stats, player_b_round_5_stats, player_b_total_stats, 

def get_fight_sig_stats(totals):
    fighter_a_head_shots_landed = totals[6].text.split("of")[0].strip()
    fighter_a_head_shots_attempted = totals[6].text.split("of")[1].strip()
    fighter_b_head_shots_landed = totals[7].text.split("of")[0].strip()
    fighter_b_head_shots_attempted = totals[7].text.split("of")[1].strip()
    fighter_a_body_shots_landed = totals[8].text.split("of")[0].strip()
    fighter_a_body_shots_attempted = totals[8].text.split("of")[1].strip()
    fighter_b_body_shots_landed = totals[9].text.split("of")[0].strip()
    fighter_b_body_shots_attempted = totals[9].text.split("of")[1].strip()
    fighter_a_leg_shots_landed = totals[10].text.split("of")[0].strip()
    fighter_a_leg_shots_attempted = totals[10].text.split("of")[1].strip()
    fighter_b_leg_shots_landed = totals[11].text.split("of")[0].strip()
    fighter_b_leg_shots_attempted = totals[11].text.split("of")[1].strip()
    fighter_a_distance_shots_landed = totals[12].text.split("of")[0].strip()
    fighter_a_distance_shots_attempted = totals[12].text.split("of")[1].strip()
    fighter_b_distance_shots_landed = totals[13].text.split("of")[0].strip()
    fighter_b_distance_shots_attempted = totals[13].text.split("of")[1].strip()
    fighter_a_clinch_landed = totals[14].text.split("of")[0].strip()
    fighter_a_clinch_attempted = totals[14].text.split("of")[1].strip()
    fighter_b_clinch_landed = totals[15].text.split("of")[0].strip()
    fighter_b_clinch_attempted = totals[15].text.split("of")[1].strip()
    fighter_a_ground_landed = totals[16].text.split("of")[0].strip()
    fighter_a_ground_attempted = totals[16].text.split("of")[1].strip()
    fighter_b_ground_landed = totals[17].text.split("of")[0].strip()
    fighter_b_ground_attempted = totals[17].text.split("of")[1].strip()

    fighter_a = fighter_a_head_shots_landed, fighter_a_head_shots_attempted, fighter_a_body_shots_landed, fighter_a_body_shots_attempted, fighter_a_leg_shots_landed, fighter_a_leg_shots_attempted, fighter_a_distance_shots_landed, fighter_a_distance_shots_attempted, fighter_a_clinch_landed, fighter_a_clinch_attempted, fighter_a_ground_landed, fighter_a_ground_attempted
    fighter_b = fighter_b_head_shots_landed, fighter_b_head_shots_attempted, fighter_b_body_shots_landed, fighter_b_body_shots_attempted, fighter_b_leg_shots_landed, fighter_b_leg_shots_attempted, fighter_b_distance_shots_landed, fighter_b_distance_shots_attempted, fighter_b_clinch_landed, fighter_b_clinch_attempted, fighter_b_ground_landed, fighter_b_ground_attempted

    return fighter_a, fighter_b

def get_fight_date_and_location(soup):
    data = soup.find_all('li', attrs={'class' : 'b-list__box-list-item'})
    date = data[0].text.strip().split("Date:")[1].strip()
    location = data[1].text.strip().split("Location:")[1].strip()
    return date, location



In [109]:
scrape_ufc_stats()